## State of art inspired mesh model

In [260]:
import pandas as pd
import numpy as np
import string
import tensorflow as tf
import os
import numpy as np
from nltk.tokenize import word_tokenize
from scipy.sparse import csr_matrix
import pandas as pd
from collections import Counter
import re
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import sklearn

In [261]:
df = pd.read_csv(r'D:\SEM-2\NLP\Project\data\hashtag_tweets.csv')
TrainX = []

In [262]:
df[:2]

,twitter_id,label,text,hashtag,name,geo,coordinates,screen_name,description
0,572342978255048705,racism,So Drasko just said he was impressed the girls...,['mkr'],thefoxbandit,NaN,NaN,thefoxbandit,"pro gamer,drawer, full time nerd and emo"
1,572341498827522049,racism,Drasko they didn't cook half a bird you idiot ...,['mkr'],patricia hilder,NaN,NaN,trish2295,NaN


In [263]:
df['text'] = df['text'].replace(np.nan, 'no_text')
df['description'] = df['description'].replace(np.nan, 'no_desc')
df['screen_name'] = df['screen_name'].replace(np.nan, 'no_name')
df['hashtag'] = df['hashtag'].replace(np.nan, 'no_hash')

In [264]:
TrainX1 = df['text']
TrainX2 = df['screen_name']
TrainX3 = df['description']
TrainX4 = df['hashtag']
TrainY = df['label']

In [265]:
from collections import Counter
Counter(TrainY)

Counter({'racism': 12,
         'sexism': 2708,
         "[{'text': 'feminists', 'indices': [49, 59]}, {'text': 'yesallmen', 'indices': [60, 70]}]": 1,
         'none': 7626})

In [266]:
TrainY_fin = np.ones(len(df))
for i in range(len(df)):
    if TrainY[i]=='none':
        TrainY_fin[i] = 0
TrainY = TrainY_fin

In [267]:
def preprocess(sent):
    sent = sent.lower()
    language = string.ascii_lowercase + string.digits + string.punctuation + ' '
    #to filter out unicode or foreign characters
    sent = "".join([char for char in sent if char in language])
    #remove all HTML tags
    sent = sent.replace("<.*?>", " ")
    #replacing multiple spaces with single space
    sent = re.sub(r"\.+(\s?\.+)*", ".", sent)
    #introducing space after punctuation if not already there
    sent = re.sub(r"([,\.\"'\?])(?!\s)", r"\1 ", sent) 
    #introducing space before punctuation if not already there
    sent = re.sub(r"(?<!\s)([,\.\"'\?])", r" \1", sent) 
    #replacing words ending in "n't" with "word not"
    sent = re.sub(r"([a-zA-z])(n't)", r"\1 not", sent) 
    #replacing multiple spaces with single space
    sent = re.sub(r"\s+", r" ", sent)
    return sent

In [268]:
Counter(TrainY_fin)

Counter({1.0: 2721, 0.0: 7626})

#### Preperation of the first type of input - Tweet text (Text data)

In [269]:
print(len(TrainX1),len(TrainY))
X = np.array(TrainX1)
Y = np.array(TrainY)

10347 10347


In [270]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

In [271]:
size_of_input = X_train.shape[0]
size_of_input

8277

In [272]:
print(len(X_train),len(X_test))

8277 2070


In [273]:
vocab = set(X_train[0])
for i in range(X_train.shape[0]):
    X_train[i] = X_train[i][:75]
    current_dict =  set(X_train[i].split())
    vocab = vocab.union(current_dict)

In [274]:
vocab_size1 = len(vocab)
embedding_dim = 10
max_length = 75
trunc_type='post'
oov_tok = "<OOV>"
print(vocab_size)

1194


In [275]:
# Tokenization
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train) 

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X_train)
testing_sequences = tokenizer.texts_to_sequences(X_test)

# Padding
padded = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_padded = tf.keras.preprocessing.sequence.pad_sequences(testing_sequences, maxlen=max_length)

In [276]:
print(f'Train padded shape: {padded.shape}')
print(f'Test padded shape: {testing_padded.shape}')

padded = tf.convert_to_tensor(padded)
testing_padded = tf.convert_to_tensor(testing_padded)


X_train=np.asarray(padded).astype(np.int)
y_train=np.asarray(y_train).astype(np.int)
X_test=np.asarray(testing_padded).astype(np.int)
y_test=np.asarray(y_test).astype(np.int)
embedding_Layer = tf.keras.layers.Embedding(vocab_size, 8)

Train padded shape: (8277, 75)
Test padded shape: (2070, 75)


In [277]:
inputA1 =  X_train
inputA_test =  X_test

#### Preperation of second type of input -  Screen name (Meta data)

In [278]:
print(len(TrainX3),len(TrainY))
X = np.array(TrainX3)
Y = np.array(TrainY)

10347 10347


In [279]:
print(X.shape,Y.shape)

(10347,) (10347,)


In [280]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

In [281]:
print(len(X_train),len(X_test))

8277 2070


In [282]:
vocab = set(X_train[0])
for i in range(X_train.shape[0]):
    X_train[i] = X_train[i][:75]
    current_dict =  set(X_train[i].split())
    vocab = vocab.union(current_dict)

In [283]:
vocab_size2 = len(vocab)
embedding_dim = 10
max_length = 75
trunc_type='post'
oov_tok = "<OOV>"
print(vocab_size)

1194


In [284]:
# Tokenization
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train) 

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X_train)
testing_sequences = tokenizer.texts_to_sequences(X_test)

# Padding
padded = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_padded = tf.keras.preprocessing.sequence.pad_sequences(testing_sequences, maxlen=max_length)

In [285]:
print(f'Train padded shape: {padded.shape}')
print(f'Test padded shape: {testing_padded.shape}')

padded = tf.convert_to_tensor(padded)
testing_padded = tf.convert_to_tensor(testing_padded)


X_train=np.asarray(padded).astype(np.int)
y_train=np.asarray(y_train).astype(np.int)
X_test=np.asarray(testing_padded).astype(np.int)
y_test=np.asarray(y_test).astype(np.int)
embedding_Layer = tf.keras.layers.Embedding(vocab_size, 8)

Train padded shape: (8277, 75)
Test padded shape: (2070, 75)


In [286]:
inputB1 =  X_train
inputB_test =  X_test

#### Preperation of third type of input: User-bio (user-data)

In [287]:
print(len(TrainX2),len(TrainY))
X = np.array(TrainX2)
Y = np.array(TrainY)

10347 10347


In [288]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

In [289]:
print(X_train.shape,y_train.shape)

(8277,) (8277,)


In [290]:
vocab = set(X_train[0])
for i in range(X_train.shape[0]):
    X_train[i] = X_train[i][:75]
    current_dict =  set(X_train[i].split())
    vocab = vocab.union(current_dict)

In [291]:
vocab_size3 = len(vocab)
embedding_dim = 10
max_length = 75
trunc_type='post'
oov_tok = "<OOV>"
print(vocab_size)

1194


In [292]:
# Tokenization
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train) 

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X_train)
testing_sequences = tokenizer.texts_to_sequences(X_test)

# Padding
padded = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_padded = tf.keras.preprocessing.sequence.pad_sequences(testing_sequences, maxlen=max_length)

In [293]:
print(f'Train padded shape: {padded.shape}')
print(f'Test padded shape: {testing_padded.shape}')

padded = tf.convert_to_tensor(padded)
testing_padded = tf.convert_to_tensor(testing_padded)


X_train=np.asarray(padded).astype(np.int)
y_train=np.asarray(y_train).astype(np.int)
X_test=np.asarray(testing_padded).astype(np.int)
y_test=np.asarray(y_test).astype(np.int)
embedding_Layer = tf.keras.layers.Embedding(vocab_size, 8)

Train padded shape: (8277, 75)
Test padded shape: (2070, 75)


In [294]:
inputC1 =  X_train
inputC_test =  X_test

#### Preperation of fourth type of input (hashtag: text data but no sequential element present)

In [295]:
print(len(TrainX4),len(TrainY))
X = np.array(TrainX4)
Y = np.array(TrainY)

10347 10347


In [296]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

In [297]:
print(X_train.shape,y_train.shape)

(8277,) (8277,)


In [298]:
vocab = set(X_train[0])
for i in range(X_train.shape[0]):
    X_train[i] = X_train[i][:75]
    current_dict =  set(X_train[i].split())
    vocab = vocab.union(current_dict)

In [299]:
vocab_size4 = len(vocab)
embedding_dim = 10
max_length = 75
trunc_type='post'
oov_tok = "<OOV>"
print(vocab_size)

1194


In [300]:
# Tokenization
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train) 

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X_train)
testing_sequences = tokenizer.texts_to_sequences(X_test)

# Padding
padded = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_padded = tf.keras.preprocessing.sequence.pad_sequences(testing_sequences, maxlen=max_length)

In [301]:
print(f'Train padded shape: {padded.shape}')
print(f'Test padded shape: {testing_padded.shape}')

padded = tf.convert_to_tensor(padded)
testing_padded = tf.convert_to_tensor(testing_padded)


X_train=np.asarray(padded).astype(np.int)
y_train=np.asarray(y_train).astype(np.int)
X_test=np.asarray(testing_padded).astype(np.int)
y_test=np.asarray(y_test).astype(np.int)
embedding_Layer = tf.keras.layers.Embedding(vocab_size, 8)

Train padded shape: (8277, 75)
Test padded shape: (2070, 75)


In [302]:
inputD1 =  X_train
inputD_test =  X_test

#### Model definition | The output from each of the models is concatenated and fed into another dense layer

In [303]:
from keras.layers import Input, Embedding, LSTM, Dense 
embedding_dim = 10

In [304]:
inputA = tf.keras.Input(shape=(75))
inputB = tf.keras.Input(shape=(75))
inputC = tf.keras.Input(shape=(75))
inputD = tf.keras.Input(shape=(75))

# the first branch operates on the first input
x1 = tf.keras.layers.Embedding(vocab_size1, embedding_dim)(inputA)
x1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim))(x1)
x1 = tf.keras.layers.Dense(4, activation="relu")(x1)
x1 = tf.keras.layers.Dense(8, activation="relu")(x1)
x1 = tf.keras.layers.Dense(16, activation="relu")(x1)
x1 = tf.keras.layers.Dense(embedding_dim, activation="relu")(x1)
x1 = tf.keras.layers.Dense(1, activation="sigmoid")(x1)
x1 = tf.keras.Model(inputs=inputA, outputs=x1)

# the second branch opreates on the second input
x2 = tf.keras.layers.Embedding(vocab_size2, embedding_dim)(inputB)
x2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim))(x2)
x2 = tf.keras.layers.Dense(4, activation="relu")(x2)
x2 = tf.keras.layers.Dense(8, activation="relu")(x2)
x2 = tf.keras.layers.Dense(16, activation="relu")(x2)
x2 = tf.keras.layers.Dense(embedding_dim, activation="relu")(x2)
x2 = tf.keras.layers.Dense(1, activation="sigmoid")(x2)
x2 = tf.keras.Model(inputs=inputB, outputs=x2)


# the third branch opreates on the second input
x3 = tf.keras.layers.Dense(64, activation="relu")(inputC)
x3 = tf.keras.layers.Dense(32, activation="relu")(x3)
x3 = tf.keras.layers.Dense(embedding_dim, activation="relu")(x3)
x3 = tf.keras.layers.Dense(1, activation="sigmoid")(x3)
x3 = tf.keras.Model(inputs=inputC, outputs=x3)


# the fourth branch opreates on the second input
x4 = tf.keras.layers.Dense(64, activation="relu")(inputD)
x4 = tf.keras.layers.Dense(32, activation="relu")(x4)
x4 = tf.keras.layers.Dense(embedding_dim, activation="relu")(x4)
x4 = tf.keras.layers.Dense(1, activation="sigmoid")(x4)
x4 = tf.keras.Model(inputs=inputD, outputs=x4)


# combine the output of the two branches
combined = tf.concat([x1.output, x2.output, x3.output, x4.output],axis=1
)

z =  tf.keras.layers.Dense(64, activation='relu')(combined)
z =  tf.keras.layers.Dense(1, activation='relu')(z)

model = tf.keras.Model(inputs=[x1.input, x2.input, x3.input, x4.input], outputs=z)
model.summary()
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

Model: "model_89"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_90 (InputLayer)           [(None, 75)]         0                                            
__________________________________________________________________________________________________
input_91 (InputLayer)           [(None, 75)]         0                                            
__________________________________________________________________________________________________
embedding_83 (Embedding)        (None, 75, 10)       202180      input_90[0][0]                   
__________________________________________________________________________________________________
embedding_84 (Embedding)        (None, 75, 10)       49010       input_91[0][0]                   
___________________________________________________________________________________________

#### Training the model

In [306]:
history = model.fit([inputA1, inputB1,inputC1,inputD1],y_train,epochs=50,)

Epoch 1/50
259/259 [==============================] - 4s 17ms/step - loss: 0.1897 - accuracy: 0.9413
Epoch 2/50
259/259 [==============================] - 4s 17ms/step - loss: 0.3534 - accuracy: 0.8823
Epoch 3/50
259/259 [==============================] - 4s 17ms/step - loss: 0.2751 - accuracy: 0.9083
Epoch 4/50
259/259 [==============================] - 4s 17ms/step - loss: 0.2208 - accuracy: 0.9307
Epoch 5/50
259/259 [==============================] - 4s 17ms/step - loss: 0.2076 - accuracy: 0.9375
Epoch 6/50
259/259 [==============================] - 4s 17ms/step - loss: 0.1945 - accuracy: 0.9444
Epoch 7/50
259/259 [==============================] - 4s 17ms/step - loss: 0.1848 - accuracy: 0.9467
Epoch 8/50
259/259 [==============================] - 4s 17ms/step - loss: 0.2070 - accuracy: 0.9385
Epoch 9/50
259/259 [==============================] - 4s 17ms/step - loss: 0.3161 - accuracy: 0.9319
Epoch 10/50
259/259 [==============================] - 4s 17ms/step - loss: 0.2896 - accura

KeyboardInterrupt: 

#### Testing the performance on unseen test data

In [307]:
metrix = model.evaluate([inputA_test,inputB_test,inputC_test,inputD_test],y_test)

65/65 [==============================] - 1s 3ms/step - loss: 0.8658 - accuracy: 0.8754


A test-accuracy of 87 percent is obtained!